# Gridsearching KNNBaseline

In [0]:
import pandas as pd
import numpy as np
import surprise as sur
import random

Read in the review dataset and prepare the dataset to be processed by the Surprise library using the Reader and Dataset classes.

In [0]:
df=pd.read_csv('df_sub.csv.gz', 
                       compression='gzip').astype({'rating':'int8', 'total_votes':'int32'})

In [0]:
reader = sur.Reader(rating_scale=(1,5))
data = sur.Dataset.load_from_df(df[['reviewerId', 'asin','rating']], reader)

In [5]:
#Splitting the data into train and test set

raw_ratings = data.raw_ratings
np.random.seed(1)
# shuffle ratings if you want
random.shuffle(raw_ratings)

threshold = int(.9 * len(raw_ratings))
A_raw_ratings = raw_ratings[:threshold]
B_raw_ratings = raw_ratings[threshold:]

print(len(A_raw_ratings))
print(len(B_raw_ratings))

data.raw_ratings = A_raw_ratings

246294
27367


Gridsearching KNNBaseline with various options. I ran through quite a few iterations of testing different ranges for all options, see the [Surprise documentation](https://surprise.readthedocs.io/en/stable/index.html) for more information.

In [0]:
param_grid = {'bsl_options': {'method': ['als', 'sgd'], 
                              'reg_i': [4, 5, 6, 7],
                              'reg_u': [8, 10, 12, 14],
                              'reg' : [0.05, 0.1, 0.2, 0.25], 
                              'n_epochs': [15],
                              'learning_rate': [0.005, 0.01]
                              },
              'sim_options': {'name': ['pearson_baseline'],
                              'user_based': [False, True], 
                              'shrinkage': [90, 100, 110]},
              'k': [30, 40, 50],
              'min_k': [1, 2]
              }
grid_search = sur.model_selection.GridSearchCV(sur.KNNBaseline,
                                               param_grid,
                                               measures=['rmse'],
                                               cv=3,
                                               refit=True)

In [25]:
grid_search.fit(data)

algo = grid_search.best_estimator['rmse']

print('Best score ', end='   ')
print(grid_search.best_score)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline si

In [26]:
#Obtaining the optimal parameters
grid_search.best_params

{'rmse': {'bsl_options': {'learning_rate': 0.005,
   'method': 'als',
   'n_epochs': 15,
   'reg': 0.2,
   'reg_i': 5,
   'reg_u': 10},
  'k': 50,
  'min_k': 2,
  'sim_options': {'name': 'pearson_baseline',
   'shrinkage': 90,
   'user_based': False}}}

In [10]:
algo = grid_search.best_estimator['rmse']

AttributeError: ignored

Refitting the optimal algo with the training and test set to compute trainign and test scores.

In [0]:
# retrain on the whole set A
trainset = data.build_full_trainset()
algo.fit(trainset)

# Compute score on training set
trainset_build = trainset.build_testset()
predictions_train = algo.test(trainset_build)
print('Training score ', end='   ')
print(sur.accuracy.rmse(predictions_train))

# Compute score on rated test set
testset = data.construct_testset(B_raw_ratings)  # testset is now the set B
predictions_test = algo.test(testset)
print('Test score (rated items) ', end=' ')
print(sur.accuracy.rmse(predictions_test))


Estimating biases using als...
Training score    RMSE: 0.8112
0.8112321010500108
Test score (rated items)  RMSE: 0.8467
0.8466962394678392
